Вариант № 5: Проверка устойчивости отношения порядка при изменении данных

Дано множество задач с дедлайнами и зависимостями (например, задача X должна быть выполнена до задачи Y).

1. Постройте отношение строгого порядка.
2. Напишите программу на Python, которая проверяет, сохраняется ли это отношение при случайном удалении 10% задач.
3. Предложите стратегию пересчета порядка при изменении данных.
4. Где  в области ИТ пригодится решение задания данного варианта?

In [15]:
import numpy as np
import random

# Задачи и их зависимости
tasks = [
    "Посмотреть фильм (19:00)",
    "Купить продукты (12:00)",
    "Почистить обувь (13:00)",
    "Отнести вещи в химчистку (14:00) - после 2",
    "Выполнить домашнюю работу (15:00) - после 3",
    "Позвонить другу (16:00)",
    "Подготовить отчет (17:00) - после 5",
    "Заниматься спортом (18:00)",
    "Встреча с друзьями (19:00) - после 6",
    "Сон (23:00)",
    "Прогулка с собакой (11:00)",
    "Заправить автомобиль (10:00)"  # добавляем две дополнительные задачи
]

# Зависимости: задача → от каких задач зависит
dependencies = {
    4: [2],
    5: [3],
    7: [5],
    9: [6],
    11: [12]  # дополнительная связь для новой задачи
}

# Функция для проверки строгого порядка
def check_properties(matrix):
    size = len(matrix)
    results = {
        'Антирефлексивность': True,
        'Антисимметричность': True,
        'Транзитивность': True
    }
    
    # Проверка антирефлексивности: отсутствие петель (задачи не зависят сами от себя)
    for i in range(size):
        if matrix[i][i]:
            results['Антирефлексивность'] = False
            break
        
    # Проверка антисимметричности: нет взаимных зависимостей
    for i in range(size):
        for j in range(i + 1, size):
            if matrix[i][j] and matrix[j][i]:
                results['Антисимметричность'] = False
                break
                
    # Проверка транзитивности: если A→B и B→C, то обязательно A→C
    for i in range(size):
        for j in range(size):
            if matrix[i][j]:  # если i зависит от j
                for k in range(size):
                    if matrix[j][k] and not matrix[i][k]:
                        results['Транзитивность'] = False
                        break
                        
    return results

# Формирование начальной матрицы зависимостей
n = len(tasks)
matrix = np.zeros((n, n))
for task, deps in dependencies.items():
    for dep in deps:
        matrix[dep-1][task-1] = 1

# Автоматически удаляем примерно 10% задач
remove_count = int(n * 0.1)  # округляем вниз
to_remove = sorted(random.sample(range(len(tasks)), remove_count))

# Сообщаем, какие задачи были удалены
removed_tasks = [tasks[idx] for idx in to_remove]
print(f"Удалённые задачи ({len(removed_tasks)} штук):\n" + '\n'.join(removed_tasks))

# Убираем выбранные задачи
tasks = [task for idx, task in enumerate(tasks) if idx not in to_remove]

# Обновляем матрицу зависимостей
matrix = np.delete(matrix, to_remove, axis=0)
matrix = np.delete(matrix, to_remove, axis=1)

# Обновляем зависимости
updated_deps = {}
for t, d in dependencies.items():
    updated_d = []
    for dep in d:
        if dep-1 not in to_remove:
            updated_d.append(dep)
    if updated_d:
        updated_deps[t] = updated_d

# Проверка свойств после удаления
props = check_properties(matrix)

# Вывод итоговых результатов
print("\nОбновленный список задач:")
for i, task in enumerate(tasks):
    print(f"{i+1}. {task}")

print("\nНовая матрица зависимостей:")
print(matrix)

print("\nСвойства после удаления:")
for p, v in props.items():
    print(f"{p:<20}: {'Да' if v else 'Нет'} ")

# Итоговый вывод
if all(props.values()):
    print("\nИтог: 👍 Строгий порядок сохраняется после удаления задач.")
else:
    print("\nИтог: 😕 Свойства строгого порядка нарушились после удаления задач.")

Удалённые задачи (1 штук):
Подготовить отчет (17:00) - после 5

Обновленный список задач:
1. Посмотреть фильм (19:00)
2. Купить продукты (12:00)
3. Почистить обувь (13:00)
4. Отнести вещи в химчистку (14:00) - после 2
5. Выполнить домашнюю работу (15:00) - после 3
6. Позвонить другу (16:00)
7. Заниматься спортом (18:00)
8. Встреча с друзьями (19:00) - после 6
9. Сон (23:00)
10. Прогулка с собакой (11:00)
11. Заправить автомобиль (10:00)

Новая матрица зависимостей:
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]

Свойства после удаления:
Антирефлексивность  : Да 
Антисимметричность  : Да 
Транзитивность      : Да 

Итог: 👍 Строгий порядок сохр

# Задание 3.
Стратегия простая:

Удалили задачу - убрали её из списка и из всех зависимостей

Перепроверили три правила:

Правило 1: Никакая задача не зависит от самой себя

Правило 2: Не должно быть взаимных зависимостей (если А перед Б, то Б не может быть перед А)

Правило 3: Если А перед Б и Б перед В, то А должно быть перед В

Если все три правила работают - порядок правильный
Если хоть одно правило нарушилось - порядок сломался

# Задание 4.

**Применение в IT: системы сборки программ**

**Контекст:** При сборке программ существует строгая последовательность обработки файлов:
- Исходные коды → Объектные файлы → Исполняемый файл

**Конкретный пример:**
1. `main.c` - исходный код
2. `utils.c` - исходный код  
3. `main.o` - зависит от `main.c`
4. `utils.o` - зависит от `utils.c`
5. `program` - зависит от `main.o` и `utils.o`

**Проблема:** При удалении `utils.c` нарушается цепочка сборки:
- `utils.o` не может быть создан
- `program` не может быть собран

**Решение:** Наша программа автоматически обнаруживает это нарушение, проверяя:
1. **Антирефлексивность** - файлы не зависят от себя
2. **Антисимметричность** - нет циклических зависимостей  
3. **Транзитивность** - сохранение цепочки зависимостей

**Практическое значение:** Такая проверка используется в:
- Системах сборки (Make, CMake)
- Пакетных менеджерах (npm, pip)
- Системах оркестрации (Kubernetes)
- Инструментах управления зависимостями

**Вывод:** Задание развивает навык анализа целостности систем, что критически важно для создания надежного программного обеспечения.